In [ ]:
library(cutpointr)
info3_area = function (ref, test, prevalence = NULL){
  ###########################################
  #############Rule-in Function##############
  ##########################################
  
  
  #############Joint Entropy Function##############
  ##########################################
  je = function(tpcm,tncm,fpcm,fncm){
    
    joint_entropy = -(tpcm*log2(tpcm))-(tncm*log2(tncm))-(fncm*log2(fncm))-(fpcm*log2(fpcm))
    
    return(joint_entropy)
    
  }
  
  
  df = data.frame(ref, test)####The data set that includes the continous biomarker-a.k.a test- and status-a.k.a ref-
  
  
  if (is.null(prevalence)) 
    
    prevalence = sum(df$ref == 1)/length(df$ref)###specifiying prevelance
  
  ###KL distances
  prev = prevalence
  M=function(prev,se,sp){
    m=prev*se+((1-prev)*(1-sp))
    return(m)
  }
  
  info.p<-function(se,sp,prev,M){
    info.<-((1/M)*se*prev*log2(se))+
      (1/M)*(1-sp)*(1-prev)*log2(1-sp)-
      log2(M)
    return(info.)
  }
  ###########################################
  ########Negatif test sonu?lu KL uzakl?gi
  info.n<-function(se,sp,prev,M){
    info..<-(1/(1-M))*(1-se)*prev*log2(1-se)+
      (1/(1-M))*(sp)*(1-prev)*log2(sp)-
      log2(1-M)
    return(info..)
  }
  n=nrow(df)
  sum=numeric(n)
  dif=numeric()
  minimum=function(x){
    for(i in 1:n){
      for(j in 1:n){
        dif[j]=(x[i]-x[j])^2
        j=j+1
      }
      sum [i]=sum(dif)
      i=i+1
      
    }
    return(sum)
  }
  
  df$distance = minimum(df$test)
  
  cut3 = df[which.min(df$distance),2]
  
  
  
  tab = as.matrix(table(df$test, df$ref))#####the counts of status at each point of test value
  
  totpos = sum(tab[, 2])#######the total number of  positive's from the status
  
  totneg = sum(tab[, 1])#######the total number of negative's from the status
  
  ###########Create the data frame from the previous the counts of status at each point of test value
  
  rdf = data.frame(thresholds = unique(sort(test)), 
                   
                   d0 = tab[, "0"], d1 = tab[, "1"], row.names = 1:nrow(tab))
  
  ###################################################################
  
  rdf$tot = rowSums(tab)####the counts of observations in the each point of the data
  
  #####Calculte the TP, FP, TN, FN, and its rates from the below
  
  rdf$TP = unname(rev(cumsum(rev(tab[, 2]))))
  
  rdf$FP = unname(rev(cumsum(rev(tab[, 1]))))
  
  rdf$TN = totneg - rdf$FP
  
  rdf$FN = totpos - rdf$TP
  
  rdf$tpr = rdf$TP/totpos
  
  rdf$tnr = rdf$TN/totneg
  
  rdf$fpr = rdf$FP/totneg
  
  rdf$fnr = rdf$FN/totpos
  
  rdf$sp = 1 - rdf$fpr
  
  rdf$se = rdf$tpr

  

  #############The proportion of TP,TN,FN,FP to all sample in order to calculate joint entropy
  
  rdf$tpcm = rdf$TP/nrow(df)
  
  rdf$fpcm = rdf$FP/nrow(df)
  
  rdf$tncm = rdf$TN/nrow(df)
  
  rdf$fncm = rdf$FN/nrow(df)
  
  ############
  
  rdf$preodds = prevalence/(1 - prevalence)####preodds
  
  rdf$neg.lr = (1 - rdf$se)/rdf$sp######Negative likelihood ratio
  
  rdf$pos.lr = rdf$se/(1 - rdf$sp)#########Positive likelihood ratio
  
  overlapping1 = min(df[which(df$ref==1),2])
  
  overlapping2 = max(df[which(df$ref==0),2])
  
  rdf$pos.prob = M(prev=prevalence,se=rdf$se, sp=rdf$sp)
  
  rdf$KL_pos = info.p (se=rdf$se, sp=rdf$sp, prev=prevalence,M=rdf$pos.prob)
  
  rdf$KL_neg = info.n (se=rdf$se, sp=rdf$sp, prev=prevalence,M=rdf$pos.prob)
  
  rdf$KL = rdf$KL_pos*rdf$pos.prob+rdf$KL_neg*(1-rdf$pos.prob)
  
  KL_pos.max=rdf[which.max(rdf$KL_pos),1]
  
  KL_neg.max=rdf[which.max(rdf$KL_neg),1]
  
  
  ########Restrict the results for the overlapped area of the two distribution
  
  rdf_reduced = rdf[which(overlapping1<= rdf$thresholds & rdf$thresholds <= overlapping2),]
  
  
  rdf_reduced$je = je(tpcm=rdf_reduced$tpcm, tncm=rdf_reduced$tncm, fpcm=rdf_reduced$fpcm,fncm=rdf_reduced$fncm)###Joint entropy
  
  ####Restrict the area between cut-of and initial maximum value
  
  up = subset(rdf_reduced, thresholds>cut3 & thresholds<overlapping2)
  
  ####Restrict the area between cut-of and initial minimum value
  
  low = subset(rdf_reduced, thresholds>overlapping1 & thresholds<cut3)

  
 #####Find the maximum value of joint entropy between cutof and initial maximum value
  
  utpos = up[which(up$je==max(up$je,na.rm = T)),1]
  
  #####Find the maximum value of joint entropy between cutof and initial minimum value
  
  ltpos = low[which(low$je==max(low$je,na.rm = T)),1]
  
  
  return(list(table = rdf_reduced,table1=rdf, thresholds = c(overlapping1,overlapping2,cut3,ltpos,utpos)))
}

